In [1]:
import netgen.gui
from netgen.geom2d import *
from ngsolve import *

In [14]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.8))
Q = L2(mesh, order=0)
ba = BitArray(mesh.ne)
print(mesh.ne)
ba.Set()
ba[0] = False
print(ba)

for v in mesh.vertices:
    print (v, v.point)

for el in mesh.Elements(VOL):
    print(type(el))
    print ("vertices: ", el.vertices)   # get vertices of an element
    print ("edges: ", el.edges)  
#ba.Set()
#ba[0] = False
#print(ba)
#Q = Compress(Q, active_dofs=ba)

4
0: 0111
V0 (0.0, 0.0)
V1 (1.0, 0.0)
V2 (1.0, 1.0)
V3 (0.0, 1.0)
V4 (0.5, 0.5000000007631067)
<class 'ngsolve.comp.Ngs_Element'>
vertices:  (V0, V1, V4)
edges:  (E2, E4, E0)
<class 'ngsolve.comp.Ngs_Element'>
vertices:  (V1, V2, V4)
edges:  (E4, E6, E3)
<class 'ngsolve.comp.Ngs_Element'>
vertices:  (V2, V3, V4)
edges:  (E6, E7, E5)
<class 'ngsolve.comp.Ngs_Element'>
vertices:  (V0, V4, V3)
edges:  (E1, E7, E2)


In [4]:
from netgen.geom2d import SplineGeometry
from netgen.meshing import MeshingParameters
from ngsolve import *                               
#from xfem import *
#from xfem.lsetcurv import *

# Geometry 
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)

#quad version:
ngmesh = square.GenerateMesh(maxh=0.15, quad_dominated=True)
mesh = Mesh (ngmesh)

#Boundary with Level set method
R=1.#Circle's  radious
levelset = sqrt(x*x+y*y) - R

Vh = H1(mesh, order = 1)  

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

lset_neg = { "levelset" : lsetp1, "domain_type" : NEG, "subdivlvl" : 0}
lset_IF = { "levelset" : lsetp1, "domain_type" : IF, "subdivlvl" : 0}

ci = CutInfo(mesh,lsetp1)
hasneg = BitArray(ci.GetElementsOfType(NEG,VOL))
hasif = BitArray(ci.GetElementsOfType(IF,VOL))#VOL or BND#

#active_dofs = GetDofsOfElements(Vh,hasneg)
#active_dofs &= Vh.FreeDofs()

NEGMesh=BitArrayCF(hasneg)
IFMesh=BitArrayCF(hasif)

Draw(NEGMesh,mesh,"NEG")
input("continue")
Draw(IFMesh,mesh,"IF")
input("continue")
test = ci.GetElementsOfType(IF,VOL)
for i in range(2):#for i in range(5):
    a0 = GetFacetsWithNeighborTypes(mesh,a=hasneg,b=test,
                                   bnd_val_a=True,bnd_val_b=True,
                                   use_and=True)
    test = GetElementsWithNeighborFacets(mesh,a0)
    Draw(BitArrayCF(test),mesh,"marked")
    print(test)
    Redraw()
    input("continue")

           
a = BilinearForm(Vh,symmetric=False)       
u,v = Vh.TrialFunction(), Vh.TestFunction()
 
# Diffusion term
print("type(IF)=",type(IF),"\n")               # <class 'xfem.ngsxfem_py.DOMAIN_TYPE'> 
print("type(lset_IF)=",type(lset_IF),"\n")     # <class 'dict'> 
print("type(test)=",type(test),"\n")           # <class 'ngsolve.ngstd.BitArray'>
#lset_IF = { "levelset" : lsetp1, "domain_type" : IF, "subdivlvl" : 0} #it works
#a += SymbolicBFI(lset_IF,form = u*v)                                  #it works
lset_test = { "levelset" : lsetp1, "domain_type" : test, "subdivlvl" : 0} #It does not give any error
print("type(lset_test)=",type(lset_test),"\n") #<class 'dict'> 
a += SymbolicBFI(lset_test,form = u*v)  

NameError: name 'LevelSetMeshAdaptation' is not defined